In [1]:
import io
import os
import json
from google.cloud import speech_v1p1beta1 as speech
from google.cloud.speech_v1p1beta1 import types

In [2]:
# set the path to your JSON key file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './gc_keys.json'


In [3]:
# create a Speech-to-Text client
client = speech.SpeechClient()

In [18]:
# set the path to your audio file
audio_file_uri = 'gs://dsc_tutorial_bucket/ep1_trimmed.mp3'

In [21]:
# configure the audio settings for the transcription request
audio = types.RecognitionAudio(uri=audio_file_uri)
config = types.RecognitionConfig(
    encoding=types.RecognitionConfig.AudioEncoding.MP3,
    sample_rate_hertz=44100,
    language_code='en-US',
    enable_speaker_diarization=True,
    diarization_speaker_count=2)

In [22]:
# perform the transcription request
response = client.long_running_recognize(config=config, audio=audio)


In [28]:
r = response.result()

In [53]:
# print the transcription results
with open('transcription.txt', 'w') as f:
    current_speaker = None
    for result in r.results:
        for word in result.alternatives[0].words:
            speaker = 'Speaker {}'.format(word.speaker_tag)
            if speaker != current_speaker:
                f.write('\n{}: '.format(speaker))
                current_speaker = speaker
            f.write('{} '.format(word.word))

In [9]:
with open('transcription.txt', 'r') as f:
    lines = f.readlines()

In [12]:
# Create an empty list to store the transformed dialogues
dialogues = []

# Loop over the lines in the transcript file and transform the dialogues
for i in range(0, len(lines), 2):
    # Extract the speakers and messages from the current dialogue
    speaker1, message1 = lines[i].strip().split(': ')
    speaker2, message2 = lines[i+1].strip().split(': ')

    # Create the prompt and completion strings for the dialogue
    prompt = f"You are an interviewer speaking in a conversational manner. Respond to the following message:{message1}"
    completion = f"{message2}"

    # Append the transformed dialogue to the dialogues list
    dialogues.append({"prompt": prompt, "completion": completion})


In [16]:
dialogues[:10]

[{'prompt': 'You are an interviewer speaking in a conversational manner. Respond to the following message:hey Max',
  'completion': 'how how you doing'},
 {'prompt': 'You are an interviewer speaking in a conversational manner. Respond to the following message:fantastic',
  'completion': 'fantastic'},
 {'prompt': 'You are an interviewer speaking in a conversational manner. Respond to the following message:I have to kind of apologize for being ridiculously happy',
  'completion': "you mean you've been inexplicably happy your whole life"},
 {'prompt': 'You are an interviewer speaking in a conversational manner. Respond to the following message:for the last maybe the last month or two in a consistent sort of manner and I attributed to various factors which we can go into are not so',
  'completion': 'thank you just spell it all out right now we can we can just shut this down and felt the inexplicable happy'},
 {'prompt': 'You are an interviewer speaking in a conversational manner. Respond 

In [17]:
with open('fine_tuning_data.json', 'w') as f:
    for pc in dialogues:
        f.write(json.dumps(pc)+'\n')
    